In [2]:
import os

os.chdir('app')

In [4]:
from prompt.store import TemplateStore

prompt_store = TemplateStore(external_store=True, ddb_table_name="prompt-store")
# prompt_store.read_from_json("prompt/database.json")
prompt_store.get_prompt_from_template("LLAMA2_CG_SYSTEM_PROMPT",{"display_name": "", "tag_name": "", "language_instructions": "", "koko":""})

' You are a helpful AI assistant that generates code.\\nYou have access to a  code interpreter environment, which supports you in your tasks.\\nYou respond by generating  code to answer user instructions.\\nThe code is executed in an interactive shell.\\nThe environment has internet, file system access and access to AWS my account.\\nTo answer instructions about my AWS account, generate  code.\\nWhen generating code, use the following format:\\n```\\n<your code>\\n```\\n\\nReport expected output and enclose it within a <output></output> tag.\\nIf you want to call  and still say something, do only output text above the code block, NOT below.\\nOnly provide at most one code block per message.\\nThe code will be executed automatically.'

In [4]:
import boto3

ddb_client = prompt_store.ddb_client
ddb_table_name = prompt_store.table_name
type(ddb_client.get_item(
    TableName=ddb_table_name, 
    Key={
        "template_id":{
            "S": "CLAUDE_CG_SYSTEM_PROMPT"
        }
    }
)["Item"])

dict

In [8]:
from prompt.template import PromptTemplate
from prompt.store import TemplateStore

prompt_store = TemplateStore()

prompt_store.add_template(
        template_id="CI_SYSTEM_PROMPT",
        template=PromptTemplate(
            template=""" You are a helpful AI assistant.

You have access to a {display_name} code interpreter, which supports you in your tasks.
The code is executed in an interactive shell, imports and variables are preserved between calls.
The environment has internet and file system access.
The current working directory is shared with the user, so files can be exchanged.
There are many libraries pre-installed, including numpy, pandas, matplotlib, and scikit-learn.
You cannot show rich outputs like plots or images, but you can store them in the working directory and point the user to them.
If the code runs too long, there will be a timeout.

To access the interpreter, use the following format:
```{tag_name}
<your code>
```
{language_instructions}
Report expected output and enclose it within a <output></output> tag.
If you want to call {display_name} and still say something, do only output text above the code block, NOT below.
Only provide at most one code block per message.
The code will be executed automatically and the result will be sent back to you
""",
            params=["display_name", "tag_name", "language_instructions"]
        )
)


prompt_store.add_template(
        template_id="CI_AGENT_REPLY",
        template=PromptTemplate(
            template=""" Thank you for the explanation. I understand I have access to a {display_name} interpreter and can execute code by placing it between ```{tag_name} tags then return expected output between <output></output> tags. I will use this to assist you with any {display_name} programming tasks. Just let me know what you need help with!
""",
            params=["display_name", "tag_name"]
        )
)


prompt_store.add_template(
        template_id="CI_SCRIPT_ERROR",
        template=PromptTemplate(
            template=" The script failed with below error, correct the script for me. Make sure to include full script between ```{tag_name} tags:\n<error>\n{error_message}\n</error>",
            params=["tag_name", "error_message"]
        )
)


prompt_store.add_template(
        template_id="CI_SCRIPT_SUCCESS",
        template=PromptTemplate(
            template=" The script executed successfully with below output, Answer with only Yes or No if this is the expected output:\n<output>\n{script_output}\n</output>",
            params=["script_output"]
        )
)

In [10]:
ROLES = ["Human", "Assistant"]

LANGUAGES = {
    "Python": {
        "display_name": "Python",
        "kernel_name": "python3",
        "tag_name": "python",
        "language_instructions": "",
        "pre_exec_script": None,
        "post_exec_script": None
    },
    "R": {
        "display_name": "R",
        "kernel_name": "ir",
        "tag_name": "R",
        "language_instructions":"",
        "pre_exec_script": None,
        "post_exec_script": None
    },
    "Java": {
        "display_name": "Java",
        "kernel_name": "java",
        "tag_name": "java",
        "language_instructions": "Always use \"Main\" as class name.",
        "pre_exec_script": None,
        "post_exec_script": "Main.main(new String[] {})"
    },
    "JavaScript": {
        "display_name": "JavaScript",
        "kernel_name": "javascript",
        "tag_name": "js",
        "language_instructions": "",
        "pre_exec_script": None,
        "post_exec_script": None
    },
    "Bash": {
        "display_name": "Bash",
        "kernel_name": "bash",
        "tag_name": "bash",
        "language_instructions": "",
        "pre_exec_script": None,
        "post_exec_script": None
    }
}

In [14]:
import json

with open("languages.json", "w") as json_f:
    json.dump(LANGUAGES, json_f, indent=4)

In [32]:
!docker context ls

NAME                              DESCRIPTION                               DOCKER ENDPOINT                                               ERROR
default                           Current DOCKER_HOST based configuration   unix:///var/run/docker.sock                                   
m5.xlarge_i-0bba463b05bcfb76e *                                             tcp://ip-172-31-78-219.ap-southeast-2.compute.internal:1111   


In [39]:
import requests
import json

docker_daemon = "ip-172-31-76-106.ap-southeast-2.compute.internal"

api_layer_addr = f"http://{docker_daemon}:8001"
api_layer_url = api_layer_addr + "/invoke"
model_family = "bedrock"
model_name = "anthropic.claude-v2"

code_executor_addr = f"http://{docker_daemon}:8080"
code_executor_url = code_executor_addr + "/execute_code"

code_block_symbol = "```"
output_tags = ["<output>", "</output>"]


def send_req_to_agent(text):
    data = {
        "body": {
            "prompt": text
        }, 
        "model_family": model_family, 
        "model_name": model_name
    }
    ret = requests.post(
        url=api_layer_url, 
        data=json.dumps(data)
    )
    return json.loads(ret.text)["generated_text"]

def send_script_to_exc(script, kernel_name):
    data = {
        "code": script, 
        "kernel_name": kernel_name
    }
    ret = requests.post(
        url=code_executor_url, 
        data=json.dumps(data)
    )
    return json.loads(ret.text)

def extract_script(text, tag_name):
    try:
        if code_block_symbol + tag_name in text:
            script = text.split(code_block_symbol + tag_name + "\n")[1].split(code_block_symbol)[0]
            expected_output = ""
            if output_tags[0] in text and output_tags[1] in text:
                expected_output = text.split(output_tags[0])[1].split(output_tags[1])[0]
            return (script, expected_output)
    except:
        return None
    return None
    

class Conversation:
    def __init__(
        self, 
        roles, 
        prompt, 
        agent, 
        executor,
        script_extractor,
        language
    ):
        self.roles = roles
        self.system_prompt = prompt
        self.history = "\n"
        self.append_chat(prompt, 0)
        self.agent = agent
        self.executor = executor
        self.script_extractor = script_extractor
        self.language = language
        self.last_agent_message = ""
        
    def append_chat(self, text, role=0):
        self.history += "\n" + self.roles[role] + ":" + text
        
    def send_to_agent(self):
        self.append_chat("", 1)
        res = self.agent(self.history)
        self.history += res
        self.last_agent_message = res
    
    def send_to_agent_and_exec_script(self):
        self.send_to_agent()
        result = self.script_extractor(
            self.last_agent_message,
            LANGUAGES[self.language]["tag_name"]
        )
        if result:
            script, expected_output = result
            output_res = ""
            if LANGUAGES[self.language]["pre_exec_script"]:
                output_res += self.executor(
                    LANGUAGES[self.language]["pre_exec_script"], 
                    LANGUAGES[self.language]["kernel_name"]
                )["output"]
            res = self.executor(script, LANGUAGES[self.language]["kernel_name"])
            res["output"] = output_res + res["output"]
            if LANGUAGES[self.language]["post_exec_script"]:
                res["output"] += self.executor(
                    LANGUAGES[self.language]["post_exec_script"], 
                    LANGUAGES[self.language]["kernel_name"]
                )["output"]
            return res
        else:
            return None
        

In [40]:
test_prompts = [
    "I want a script to print 'Hello World!'",
    # "I want a script that compute first 10 odd numbers and print them.",
    # "I want a script that compute factorial of 20 and determines the time it took to calculate the result.",
    # "I want to make tetris game"
]

language = "Java"
for language in LANGUAGES:

    params = {
        "display_name": LANGUAGES[language]["display_name"],
        "tag_name": LANGUAGES[language]["tag_name"],
        "error_message": "",
        "script_output": "",
        "language_instructions": LANGUAGES[language]["language_instructions"]
    }

    for test in test_prompts:
        conv = Conversation(
            ROLES, 
            prompt_store.get_prompt_from_template(
                "CI_SYSTEM_PROMPT",
                params
            ),
            send_req_to_agent,
            send_script_to_exc,
            extract_script,
            language
        )
        conv.append_chat(
            prompt_store.get_prompt_from_template(
                "CI_AGENT_REPLY",
                params
            ),
            1
        )
        conv.append_chat(test)
        res = conv.send_to_agent_and_exec_script()
        max_iterations = 3
        i = 0
        while i < max_iterations and res["error"]:
            i += 1
            params["error_message"] = res["output"]
            conv.append_chat(
                prompt_store.get_prompt_from_template(
                    "CI_SCRIPT_ERROR",
                    params
                )
            )
            conv.send_to_agent_and_exec_script()   
        if not res["error"]:
            params["script_output"] = res["output"]
            conv.append_chat(
                prompt_store.get_prompt_from_template(
                    "CI_SCRIPT_SUCCESS",
                    params
                )
            )
            conv.send_to_agent()
        print(conv.history)



Human: You are a helpful AI assistant.

You have access to a Python code interpreter, which supports you in your tasks.
The code is executed in an interactive shell, imports and variables are preserved between calls.
The environment has internet and file system access.
The current working directory is shared with the user, so files can be exchanged.
There are many libraries pre-installed, including numpy, pandas, matplotlib, and scikit-learn.
You cannot show rich outputs like plots or images, but you can store them in the working directory and point the user to them.
If the code runs too long, there will be a timeout.

To access the interpreter, use the following format:
```python
<your code>
```

Report expected output and enclose it within a <output></output> tag.
If you want to call Python and still say something, do only output text above the code block, NOT below.
Only provide at most one code block per message.
The code will be executed automatically and the result will be sent

In [54]:
conv.history

'\n\nHuman: You are a helpful AI assistant.\n\nYou have access to a R code interpreter, which supports you in your tasks.\nThe code is executed in an interactive shell, imports and variables are preserved between calls.\nThe environment has internet and file system access.\nThe current working directory is shared with the user, so files can be exchanged.\nThere are many libraries pre-installed, including numpy, pandas, matplotlib, and scikit-learn.\nYou cannot show rich outputs like plots or images, but you can store them in the working directory and point the user to them.\nIf the code runs too long, there will be a timeout.\n\nTo access the interpreter, use the following format:\n```R\n<your code>\n```\nIf you want to call R and still say something, do only output text above the code block, NOT below.\nOnly provide at most one code block per message.\nThe code will be executed automatically and the result will be sent back to you\n\nAssistant: Thank you for the explanation. I underst

In [9]:
!docker build --quiet --tag controller .

sha256:de91d465309ac442e5d83f226bfb753809bb45a0da0482398ebd3614bb8f029f


In [3]:
!docker context ls

NAME                             DESCRIPTION                               DOCKER ENDPOINT                                      ERROR
default                          Current DOCKER_HOST based configuration   unix:///var/run/docker.sock                          
m5.large_i-029ae266049386ca1 *                                             tcp://ip-10-3-1-85.us-west-2.compute.internal:1111   


In [18]:
host_ip1 = "internal-codenator-899847730.us-west-2.elb.amazonaws.com"
host_ip = "ip-10-3-1-85.us-west-2.compute.internal"

In [63]:
%%bash -s $host_ip -s $host_ip1
echo ${3}
docker run -d -p 8001:8001 -e AWS_REGION=us-west-2 -e AWS_DEFAULT_REGION=us-west-2 --name test-controller controller --host 0.0.0.0 --port 8001 --api-layer-host ${3} --api-layer-port 8010 --code-scanner-host ${3} --code-scanner-port 8011 --code-executor-host ${1} --code-executor-port 8080 --prompt-store-name prompt-store --models-metadata-db model-db --kms alias/codenator-kms

internal-codenator-899847730.us-west-2.elb.amazonaws.com
2e8c14523c7a8310e3e0875403e1feb758a03cc4940656a8c331a32a2ba624cb


In [60]:
!docker logs test-controller

INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)
INFO:     10.3.1.27:33656 - "POST /generate HTTP/1.1" 200 OK
{'output': 'Hello World', 'error': False, 'script': 'print("Hello World")', 'expected_output': 'Hello World', 'conv_id': '6d6ddcbed25f4d3e'}
INFO:     10.3.1.27:51740 - "POST /execute HTTP/1.1" 200 OK


In [184]:
host_ip = "ip-10-3-1-89.us-west-2.compute.internal"

In [7]:
!docker kill test-controller && docker rm test-controller

Error response from daemon: Cannot kill container: test-controller: No such container: test-controller


In [8]:
!docker rm test-controller

Error response from daemon: No such container: test-controller


In [64]:
%%bash -s $host_ip
curl http://${1}:8001/ping

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    22  100    22    0     0   3314      0 --:--:-- --:--:-- --:--:--  3666


{"Health_Check":"200"}

In [65]:
%%bash -s $host_ip
curl http://${1}:8001/list_languages

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1154  100  1154    0     0  18895      0 --:--:-- --:--:-- --:--:-- 19233


"{\"Python\": {\"display_name\": \"Python\", \"kernel_name\": \"python3\", \"tag_name\": \"python\", \"language_instructions\": \"\", \"pre_exec_script\": null, \"post_exec_script\": null}, \"R\": {\"display_name\": \"R\", \"kernel_name\": \"ir\", \"tag_name\": \"R\", \"language_instructions\": \"\", \"pre_exec_script\": null, \"post_exec_script\": null}, \"Java\": {\"display_name\": \"Java\", \"kernel_name\": \"java\", \"tag_name\": \"java\", \"language_instructions\": \"Always use \\\"Main\\\" as class name.\", \"pre_exec_script\": null, \"post_exec_script\": \"Main.main(new String[] {})\"}, \"JavaScript\": {\"display_name\": \"JavaScript\", \"kernel_name\": \"javascript\", \"tag_name\": \"js\", \"language_instructions\": \"\", \"pre_exec_script\": null, \"post_exec_script\": null}, \"Bash\": {\"display_name\": \"Bash\", \"kernel_name\": \"bash\", \"tag_name\": \"bash\", \"language_instructions\": \"\", \"pre_exec_script\": null, \"post_exec_script\": null}, \"Shell\": {\"display_nam

In [66]:
%%bash -s $host_ip
curl http://${1}:8001/generate -d '{"prompt": "Write Hello World code", "model_family": "bedrock", "model_name": "anthropic.claude-v2", "language": "Python", "stream": true}'

#'{"prompt": "Write Hello World code", "model_family": "bedrock", "model_name": "anthropic.claude-v2", "language": "Python"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847    0   709  100   138    417     81  0:00:01  0:00:01 --:--:--   499


{"script": "", "expected_output": "", "generated_text": " Here", "text": " Here", "conv_id": "3524612583f3480f"}
{"script": "print(\"Hello World", "expected_output": "", "generated_text": " Here is some Python code to print \"Hello World\":\n\n```python\nprint(\"Hello World", "text": " is some Python code to print \"Hello World\":\n\n```python\nprint(\"Hello World", "conv_id": "3524612583f3480f"}
{"script": "print(\"Hello World\")", "expected_output": "Hello World", "generated_text": " Here is some Python code to print \"Hello World\":\n\n```python\nprint(\"Hello World\")\n```\n\n<output>\nHello World\n</output>", "text": "\")\n```\n\n<output>\nHello World\n</output>", "conv_id": "3524612583f3480f"}


In [67]:
%%bash -s $host_ip
curl http://${1}:8001/execute -d '{"script": "print(\"Hello World\")", "expected_output":"Hello World", "model_family": "bedrock", "model_name": "anthropic.claude-v2", "language": "Python", "conv_id":"3524612583f3480f", "stream": true}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   344    0   143  100   201    133    187  0:00:01  0:00:01 --:--:--   320


{"output": "Hello World", "error": false, "script": "print(\"Hello World\")", "expected_output": "Hello World", "conv_id": "3524612583f3480f"}


In [45]:
%%bash -s $host_ip
curl http://${1}:8001/scan -d '{"script": "print(\"Hello World\")", "expected_output":"Hello World", "model_family": "bedrock", "model_name": "anthropic.claude-v2", "language": "Python", "conv_id":"ed522349a8de4782", "stream": true}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to ip-10-3-1-85.us-west-2.compute.internal port 8000 after 2 ms: Connection refused


CalledProcessError: Command 'b'curl http://${1}:8000/scan -d \'{"script": "print(\\\\"Hello World\\\\")", "expected_output":"Hello World", "model_family": "bedrock", "model_name": "anthropic.claude-v2", "language": "Python", "conv_id":"ed522349a8de4782", "stream": true}\'\n'' returned non-zero exit status 7.

In [106]:
%%bash -s $host_ip
curl http://${1}:8001/scan -d '{"script": "print(\"Hello World\")", "model_family": "bedrock", "model_name": "anthropic.claude-v2", "language": "Python", "conv_id": "34adc0d71a564aae"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   238  100    85  100   153     16     28  0:00:05  0:00:05 --:--:--    20


{"vulnerabilities":[],"script":"print(\"Hello World\")","conv_id":"34adc0d71a564aae"}

In [150]:
%%bash -s $host_ip
curl http://${1}:8080/generate -d '{"prompt": "Use the below script, add Hello world print message to the last line.\n```python\nfrom flask import Flask, request\napp = Flask(__name__)\n@app.route(\"/unsafe_query\", methods=[\"GET\"])\ndef unsafe_query():\n    user_id = request.args.get(\"user_id\")\n    os.system(f\"echo {user_id}\")\n\nif __name__ == \"__main__\":\n    app.run(debug=True)\n```", "model_family": "bedrock", "model_name": "anthropic.claude-v2", "language": "Python", "stream": true}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   467    0     0  100   467      0     38  0:00:12  0:00:12 --:--:--     0

Process is interrupted.


In [102]:
import boto3
import json

models_metadata_db = "models-db"

ddb_client = boto3.client("dynamodb")
ret = ddb_client.get_item(
    TableName=models_metadata_db,
    Key={
        "pk": {
            "S": "languages"
        },
        "sk": {
            "S": "languages"
        }
    }
)["Item"]["data"]["S"]
print(json.loads(ret)["JavaScript"])

{'display_name': 'JavaScript', 'kernel_name': 'javascript', 'tag_name': 'js', 'language_instructions': '', 'pre_exec_script': None, 'post_exec_script': None}


In [146]:
"key".format(**{"language": "python"})

'key'

In [78]:
!docker images

REPOSITORY      TAG       IMAGE ID       CREATED             SIZE
controller      latest    afe70ea65b75   About an hour ago   237MB
<none>          <none>    931a9fa402fe   3 hours ago         235MB
code-executor   latest    0e3269ad5dc3   3 hours ago         7.21GB
api-layer       latest    0399a50f1f6b   3 hours ago         236MB


In [79]:
!docker ps --all

CONTAINER ID   IMAGE                  COMMAND                  CREATED             STATUS                   PORTS                              NAMES
2b0dc9ab993b   controller             "python3 /code/app/m…"   About an hour ago   Up About an hour         0.0.0.0:8000->8000/tcp             test-controller
748998feca14   api-layer:latest       "python3 /code/app/m…"   2 hours ago         Up 2 hours               0.0.0.0:8001->8001/tcp             test-api-layer
28521fe9da3c   931a9fa402fe           "python3 /code/app/m…"   3 hours ago         Exited (1) 3 hours ago                                      magical_dubinsky
84a36276d3e2   931a9fa402fe           "python3 /code/app/m…"   3 hours ago         Created                                                     beautiful_bhabha
ce38cebb9925   code-executor:latest   "python3 /code/app/m…"   3 hours ago         Up 3 hours (unhealthy)   0.0.0.0:8080->8080/tcp, 8888/tcp   test


In [80]:
!docker rm 28521fe9da3c 84a36276d3e2

28521fe9da3c
84a36276d3e2


In [81]:
!docker rmi 931

Deleted: sha256:931a9fa402fe2658ffcaf29d4496b9c926a70688040e6b9579421b7b1279d12c


In [16]:
import time

def fun(stream=False):
    def iter_func(data):
        for cha in data:
            time.sleep(0.2)
            yield cha
        
    data="some data"
    if stream:
        return iter_func(data)
    else:
        return data
    
print(fun())

for item in fun(True):
    
    print(item, end="")

some data
some data

In [ ]:
host_ip = "ip-172-31-75-227.ap-southeast-2.compute.internal"

In [ ]:
api_layer_url = f"http://{host_ip}:8001/invoke"

In [26]:
results = [
    {"Number": 1},
    {"Number": 2},
    {"Number": 3},
    {"Number": 4},
    {"Number": 5},
    {"Number": 6}
]

class test:
    last_num = ""
    sum_ = 0
    def __init__(self, data):
        def iter_func(data):
            for item in data:
                self.last_num += str(item["Number"])
                self.sum_ += item["Number"]
                yield item["Number"]
        self.value = iter_func(data), self.last_num, self.sum_
    
d = test(results)
for item in d.value[0]:
    print(item, d.last_num, d.sum_)
            

1 1 1
2 12 3
3 123 6
4 1234 10
5 12345 15
6 123456 21


In [5]:
import boto3

sagemaker = boto3.client('sagemaker', region_name='ap-southeast-2')
response = sagemaker.list_models()